# Extracción

Extracción de datos de la base de datos alojada en en proyecto en Bigquery

In [6]:
# !pip install google-cloud-bigquery
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

### Creación de cliente en Google Colab  
Las credenciales de cliente se cargan a través de una interfaz que pide permisos de la cuenta de Google, de forma rápida, cómoda y compatible con Google Colab

In [7]:
# # Crear cliente en Colab
# from google.colab import auth
# from google.cloud import bigquery

# auth.authenticate_user()
# client = bigquery.Client(project="the-look-ecommerce-project")

### Creación de cliente en local 
Se obtienen las credenciales de cliente a través de una clave dada por Bigquery y configurada como variable de entorno.

In [8]:
# Crear cliente en local
from google.cloud import bigquery
import db_dtypes

client = bigquery.Client()
print("Autenticación correcta. Proyecto:", client.project)

Autenticación correcta. Proyecto: the-look-ecommerce-project


### Consulta  
Petición a la base de datos en SQL para obtener las tablas en formato DataFrame

In [9]:
# Consulta

query = """
    SELECT *
    FROM `the-look-ecommerce-project.TheLook_ecommerce.INFORMATION_SCHEMA.TABLES`;
"""

# tablas = bigquery.QueryJobConfig()
# df = client.query(query, job_config=job_config, location="us").to_dataframe()

tablas = client.query(query).to_dataframe()
display(tablas)

,table_catalog,table_schema,table_name,table_type,is_insertable_into,is_typed,creation_time,base_table_catalog,base_table_schema,base_table_name,...,default_collation_name,upsert_stream_apply_watermark,replica_source_catalog,replica_source_schema,replica_source_name,replication_status,replication_error,is_change_history_enabled,sync_status,is_fine_grained_mutations_enabled
0,the-look-ecommerce-project,TheLook_ecommerce,users,BASE TABLE,YES,NO,2025-06-28 12:24:03.743000+00:00,None,None,None,...,NULL,NaT,None,None,None,None,None,NO,None,NO
1,the-look-ecommerce-project,TheLook_ecommerce,order_items,BASE TABLE,YES,NO,2025-06-28 12:23:56.223000+00:00,None,None,None,...,NULL,NaT,None,None,None,None,None,NO,None,NO
2,the-look-ecommerce-project,TheLook_ecommerce,distribution_centers,BASE TABLE,YES,NO,2025-07-20 01:11:43.576000+00:00,None,None,None,...,NULL,NaT,None,None,None,None,None,NO,None,NO
3,the-look-ecommerce-project,TheLook_ecommerce,inventory_items,BASE TABLE,YES,NO,2025-06-28 12:23:52.462000+00:00,None,None,None,...,NULL,NaT,None,None,None,None,None,NO,None,NO
4,the-look-ecommerce-project,TheLook_ecommerce,products,BASE TABLE,YES,NO,2025-06-28 12:24:00.676000+00:00,None,None,None,...,NULL,NaT,None,None,None,None,None,NO,None,NO
5,the-look-ecommerce-project,TheLook_ecommerce,orders,BASE TABLE,YES,NO,2025-06-28 12:23:58.615000+00:00,None,None,None,...,NULL,NaT,None,None,None,None,None,NO,None,NO
6,the-look-ecommerce-project,TheLook_ecommerce,events,BASE TABLE,YES,NO,2025-07-20 17:05:30.019000+00:00,None,None,None,...,NULL,NaT,None,None,None,None,None,NO,None,NO


In [10]:
lista_tablas = tablas['table_name'].unique().tolist()
lista_tablas

['users',
 'order_items',
 'distribution_centers',
 'inventory_items',
 'products',
 'orders',
 'events']

### Preparación  
Se necesita obtener un dataframe para cada tabla de la base de datos

In [11]:
# Descargar cada tabla como diccionario de DataFrames
dataframes = {}

for tabla in lista_tablas:
    print(f"Descargando tabla: {tabla}...")
    query = f"""
        SELECT *
        FROM `the-look-ecommerce-project.TheLook_ecommerce.{tabla}`
    """
    df = client.query(query).to_dataframe()
    dataframes[tabla] = df

Descargando tabla: users...
Descargando tabla: order_items...
Descargando tabla: distribution_centers...
Descargando tabla: inventory_items...
Descargando tabla: products...
Descargando tabla: orders...
Descargando tabla: events...
{'users':           id first_name last_name                       email  age gender  \
0      85010      Jason  Phillips   jasonphillips@example.org   69      M   
1       6633     Martin    Abbott    martinabbott@example.net   63      M   
2      95190     Walter    Palmer    walterpalmer@example.com   68      M   
3       6136    Suzanne   Jackson  suzannejackson@example.net   29      F   
4       6241   Michelle    Fields  michellefields@example.com   37      F   
...      ...        ...       ...                         ...  ...    ...   
99995  35885      Colin   Sanchez    colinsanchez@example.net   67      M   
99996  42805     Darius   Andrews   dariusandrews@example.org   24      M   
99997  41311    Cynthia    Ortega   cynthiaortega@example.org   6

In [12]:
print(list(dataframes.keys()))
display(dataframes)

['users', 'order_items', 'distribution_centers', 'inventory_items', 'products', 'orders', 'events']


{'users':           id first_name last_name                       email  age gender  \
 0      85010      Jason  Phillips   jasonphillips@example.org   69      M   
 1       6633     Martin    Abbott    martinabbott@example.net   63      M   
 2      95190     Walter    Palmer    walterpalmer@example.com   68      M   
 3       6136    Suzanne   Jackson  suzannejackson@example.net   29      F   
 4       6241   Michelle    Fields  michellefields@example.com   37      F   
 ...      ...        ...       ...                         ...  ...    ...   
 99995  35885      Colin   Sanchez    colinsanchez@example.net   67      M   
 99996  42805     Darius   Andrews   dariusandrews@example.org   24      M   
 99997  41311    Cynthia    Ortega   cynthiaortega@example.org   60      F   
 99998  55447    Michele      Berg     micheleberg@example.com   26      F   
 99999  53233     Sherri   Johnson   sherrijohnson@example.net   68      F   
 
                state                 street_address 

### Carga de datos  
Se almacenan las tablas en formato csv dentro del directorio del proyecto

In [15]:
# Se obtienen los diferentes dataframes por separado y se guardan en csv

users = dataframes.get('users')
users.to_csv("../data/users.csv", index=False)

order_items = dataframes.get('order_items')
order_items.to_csv("../data/order_items.csv", index=False)

distribution_centers = dataframes.get('distribution_centers')
distribution_centers.to_csv("../data/distribution_centers.csv", index=False)

inventory_items = dataframes.get('inventory_items')
inventory_items.to_csv("../data/inventory_items.csv", index=False)

products = dataframes.get('products')
products.to_csv("../data/products.csv", index=False)

orders = dataframes.get('orders')
orders.to_csv("../data/orders.csv", index=False)

events = dataframes.get('events')
events.to_csv("../data/events.csv", index=False)